In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [3]:
data = pd.read_csv(r'..\Datasets\Parkinsons Processed Training - Only Vowels - Sri Lankan.csv', header = 0)
data.head()

,Subject Id,Jitter(local),"Jitter(local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, db)",Shimmer (apq3),Shimmer (apq5),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,status
0,1,0.01488,0.000090,0.00900,0.00794,0.02699,0.08334,0.779,0.04517,0.04609,...,187.576,160,159,0.006065,0.000416,0.00000,0,0.00000,23,1
1,1,0.00728,0.000038,0.00353,0.00376,0.01059,0.05864,0.642,0.02058,0.03180,...,234.505,170,169,0.005181,0.000403,0.02247,0,0.00000,23,1
2,1,0.00472,0.000031,0.00224,0.00245,0.00673,0.06404,0.629,0.03434,0.03765,...,190.019,675,673,0.006649,0.000132,0.26180,1,0.26325,23,1
3,2,0.00427,0.000034,0.00243,0.00249,0.00728,0.07428,0.694,0.04205,0.04311,...,129.205,345,344,0.008073,0.000143,0.00000,0,0.00000,8,1
4,2,0.00844,0.000069,0.00509,0.00489,0.01527,0.14053,1.239,0.06131,0.07015,...,126.788,585,582,0.008174,0.000123,0.00209,2,0.00768,8,1


In [4]:
X = data.iloc[:, 1:24]
X

,Jitter(local),"Jitter(local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, db)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,HTM,Median Pitch,Mean Pitch,Standard deviation,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period
0,0.01488,0.000090,0.00900,0.00794,0.02699,0.08334,0.779,0.04517,0.04609,0.06802,...,11.130,166.533,164.781,10.421,142.229,187.576,160,159,0.006065,0.000416
1,0.00728,0.000038,0.00353,0.00376,0.01059,0.05864,0.642,0.02058,0.03180,0.07194,...,17.403,195.252,193.289,14.773,159.515,234.505,170,169,0.005181,0.000403
2,0.00472,0.000031,0.00224,0.00245,0.00673,0.06404,0.629,0.03434,0.03765,0.05138,...,17.809,150.215,150.379,3.027,141.083,190.019,675,673,0.006649,0.000132
3,0.00427,0.000034,0.00243,0.00249,0.00728,0.07428,0.694,0.04205,0.04311,0.05250,...,15.267,124.171,123.883,2.179,118.461,129.205,345,344,0.008073,0.000143
4,0.00844,0.000069,0.00509,0.00489,0.01527,0.14053,1.239,0.06131,0.07015,0.09260,...,10.747,122.220,122.381,1.650,118.020,126.788,585,582,0.008174,0.000123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.00601,0.000031,0.00316,0.00339,0.00947,0.07492,0.681,0.03729,0.04581,0.06352,...,16.770,202.463,193.859,28.380,99.593,214.486,215,213,0.005161,0.001022
116,0.00382,0.000017,0.00169,0.00203,0.00508,0.06905,0.648,0.02728,0.03968,0.06920,...,19.500,221.165,222.804,6.486,205.108,237.431,188,187,0.004490,0.000134
117,0.00471,0.000034,0.00186,0.00230,0.00558,0.02793,0.260,0.01183,0.01520,0.02576,...,22.797,138.697,139.265,2.273,132.744,145.840,176,175,0.007179,0.000130
118,0.04319,0.000317,0.02774,0.01708,0.08322,0.13924,1.195,0.08558,0.06036,0.08585,...,12.266,136.805,136.026,4.846,124.543,165.029,258,256,0.007343,0.000312


In [5]:
X.shape

(120, 23)

In [6]:
y = data.iloc[:, -1]
y

0      1
1      1
2      1
3      1
4      1
      ..
115    0
116    0
117    0
118    0
119    0
Name: status, Length: 120, dtype: int64

In [7]:
standard_X = StandardScaler()
X_scaled_standard = standard_X.fit_transform(X)
X_scaled_standard

array([[ 0.9687889 ,  0.85294018,  1.1180386 , ..., -0.68478116,
        -0.13624555,  0.08863683],
       [-0.1395651 , -0.32273081, -0.18275288, ..., -0.65082366,
        -0.67139215,  0.05561585],
       [-0.5129054 , -0.46433071, -0.48952089, ...,  1.06063402,
         0.21774015, -0.64548196],
       ...,
       [-0.51436376, -0.40916832, -0.57988666, ..., -0.63044917,
         0.53857237, -0.65143669],
       [ 5.09740757,  5.93282836,  5.57449787, ..., -0.35539347,
         0.63776328, -0.18096908],
       [ 1.04899873,  1.14719933,  1.36535544, ...,  0.21848817,
         0.24858338, -0.70701073]])

In [8]:
X_scaled_standard.shape

(120, 23)

In [ ]:
cols = list(X.columns)
X_1 = X[cols]
X_1

In [ ]:
X_1.columns.size

In [ ]:
cols

In [ ]:
X_1.shape

In [ ]:
#Backward Elimination
#cols = list(X.columns)
pmax = 1
X_1 = standard_X.transform(X_1)
while (len(cols)>0):
    p= []
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
cols = list(X.columns)
model = XGBClassifier()
#Initializing RFE model
rfe = RFE(model)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)
for i in range(len(cols)):
    print(cols[i], "----->", rfe.support_[i])

In [ ]:
#no of features
nof_list=np.arange(1,24)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    #X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.05, random_state = 12)
    model = XGBClassifier()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X,y)
    #X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y)
    accuracy = cross_val_score(estimator = model, X = X, y = y, cv = 10)
    score = accuracy.mean()
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
score_list

In [ ]:
cols = list(X.columns)
model = XGBClassifier()
#Initializing RFE model
rfe = RFE(model, 10)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
X_from_rfe = data[selected_features_rfe].values

In [ ]:
X_from_rfe

In [ ]:
standard_X = StandardScaler()
X_from_rfe = standard_X.fit_transform(X_from_rfe)
X_from_rfe

In [50]:
param_grid = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
             "max_depth"        : [ 3, 4, 5, 6, 8, 10],
             "min_child_weight" : [ 1, 3, 5, 7 ],
             "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
             "n_estimators" : [10, 20, 50, 70, 100]}

In [37]:
param_grid = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] }

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=12, test_size = 0.01)

In [52]:
from sklearn.model_selection import GridSearchCV
#create new a knn model
model = XGBClassifier()
#use gridsearch to test all values for n_neighbors
model_xg = GridSearchCV(model, param_grid, cv=5, n_jobs = -1, verbose = 1)
#fit model to data
model_xg.fit(X_train, Y_train)

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 3200 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 6000 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 9600 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done 14000 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 14400 out of 14400 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
                         'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
                         'max_depth

In [53]:
model_xg.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.25,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 20}

In [54]:
model_xg.best_score_

0.727536231884058